In [3]:
from bs4 import BeautifulSoup
import uuid
import re
import json
import html
import pickle

In [4]:
def generate_uuid():
    return str(uuid.uuid4())

In [5]:
def extract_feed_data(page_source):
    try:
        # Load HTML content using BeautifulSoup
        soup = BeautifulSoup(page_source, 'html.parser')
    except Exception as e:
        print(f"Error parsing page source with BeautifulSoup: {e}")
        return {}
    
    # Result dictionaries
    posts = []
    tributes = []
    
    try:
        # Extract all feedList entries
        feed_entries = soup.find_all('div', class_='jsx-a331cc0c0f6f74f1 feedList')
    except Exception as e:
        print(f"Error finding feedList entries: {e}")
        return {}
    
    for entry in feed_entries:
        try:
            # Identify and process posts
            post_tag = entry.find('article', {'data-qa': lambda x: x and x.startswith('journalPost')})
            if post_tag:
                post_id = generate_uuid()
                author_tag = post_tag.find('div', class_='postAuthor')
                author = author_tag.get_text(strip=True) if author_tag else "Unknown Author"
                post_date_tag = post_tag.find('div', class_='postDate')
                post_date = post_date_tag.get_text(strip=True) if post_date_tag else "Unknown Date"
                post_title_tag = post_tag.find('h1', class_='postTitle')
                post_title = post_title_tag.get_text(strip=True) if post_title_tag else "Untitled"
                comments_count_tag = post_tag.find('button', {'data-qa': lambda x: x and x.startswith('toggleComments')})
                comments_count_text = comments_count_tag.get_text(strip=True) if comments_count_tag else '0 comments'
                if comments_count_text and any(char.isdigit() for char in comments_count_text):
                    comments_count = int(comments_count_text.split(' ')[0])
                else:
                    comments_count = 0
                
                # Update to correctly extract reactions count using regex to handle different formats
                reactions_count_tag = post_tag.find('button', {'data-qa': lambda x: x and x.startswith('reactionCounter')})
                reactions_count_span = reactions_count_tag.find('span', {'data-qa': 'rc-count'}) if reactions_count_tag else None
                if reactions_count_span:
                    reactions_count_text = reactions_count_span.get_text(strip=True)
                    if reactions_count_text and any(char.isdigit() for char in reactions_count_text):
                        reactions_count = int(''.join(filter(str.isdigit, reactions_count_text)))
                    else:
                        reactions_count = 0
                else:
                    reactions_count = 0
                
                post_body_tag = post_tag.find('div', {'data-qa': lambda x: x and x.startswith('postBody')})
                post_body = post_body_tag.get_text(strip=True) if post_body_tag else ""
                pictures = [img.get('src') for img in post_tag.find_all('img', {'data-nimg': '1'})]
                
                # Extract comments and their reactions
                comments = []
                comment_tags = post_tag.find_all('div', {'data-qa': 'comment'})
                for comment_tag in comment_tags:
                    commenter_tag = comment_tag.find('span', {'data-qa': lambda x: x and x.startswith('commenter')})
                    commenter_text = commenter_tag.get_text(strip=True) if commenter_tag else "Unknown Commenter"
                    if '—' in commenter_text:
                        commenter, comment_date = map(str.strip, commenter_text.split('—', 1))
                    else:
                        commenter, comment_date = commenter_text, "Unknown Date"
                    comment_body_tag = comment_tag.find('div', {'data-qa': lambda x: x and x.startswith('commentBody')})
                    comment_body = comment_body_tag.get_text(strip=True) if comment_body_tag else ""
                    
                    # Extract reactions to comments
                    comment_reactions_tag = comment_tag.find('button', {'data-qa': lambda x: x and x.startswith('reactionCounter')})
                    comment_reactions_span = comment_reactions_tag.find('span', {'data-qa': 'rc-count'}) if comment_reactions_tag else None
                    if comment_reactions_span:
                        comment_reactions_text = comment_reactions_span.get_text(strip=True)
                        if comment_reactions_text and any(char.isdigit() for char in comment_reactions_text):
                            comment_reactions_count = int(''.join(filter(str.isdigit, comment_reactions_text)))
                        else:
                            comment_reactions_count = 0
                    else:
                        comment_reactions_count = 0
                    
                    # Extract replies to comments
                    replies = []
                    reply_tags = comment_tag.find_all('div', {'data-qa': 'reply'})
                    for reply_tag in reply_tags:
                        reply_author_tag = reply_tag.find('span', {'data-qa': lambda x: x and x.startswith('commenter')})
                        reply_author_text = reply_author_tag.get_text(strip=True) if reply_author_tag else "Unknown Reply Author"
                        if '—' in reply_author_text:
                            reply_author, reply_date = map(str.strip, reply_author_text.split('—', 1))
                        else:
                            reply_author, reply_date = reply_author_text, "Unknown Date"
                        reply_body_tag = reply_tag.find('div', {'data-qa': lambda x: x and x.startswith('commentBody')})
                        reply_body = reply_body_tag.get_text(strip=True) if reply_body_tag else ""
                        
                        # Extract reactions to replies
                        reply_reactions_tag = reply_tag.find('button', {'data-qa': lambda x: x and x.startswith('reactionCounter')})
                        reply_reactions_span = reply_reactions_tag.find('span', {'data-qa': 'rc-count'}) if reply_reactions_tag else None
                        if reply_reactions_span:
                            reply_reactions_text = reply_reactions_span.get_text(strip=True)
                            if reply_reactions_text and any(char.isdigit() for char in reply_reactions_text):
                                reply_reactions_count = int(''.join(filter(str.isdigit, reply_reactions_text)))
                            else:
                                reply_reactions_count = 0
                        else:
                            reply_reactions_count = 0
                        
                        replies.append({
                            'reply_author': reply_author,
                            'reply_date': reply_date,
                            'reply_body': reply_body,
                            'reactions_count': reply_reactions_count
                        })
                    
                    comments.append({
                        'commenter': commenter,
                        'comment_date': comment_date,
                        'comment_body': comment_body,
                        'reactions_count': comment_reactions_count,
                        'replies': replies
                    })
                
                # Store post data
                posts.append({
                    'post_id': post_id,
                    'author': author,
                    'post_date': post_date,
                    'post_title': post_title,
                    'comments_count': comments_count,
                    'reactions_count': reactions_count,
                    'pictures': pictures,
                    'post_body': post_body,
                    'comments': comments
                })
                continue

            # Identify and process tributes
            signature_tag = entry.find('div', class_='jsx-50a7abfcbb14ee96 tributeSignature')
            date_tag = entry.find('div', class_='jsx-50a7abfcbb14ee96 tributeDate')
            if signature_tag and date_tag:
                #print("Debug: Tribute signature and date tags found")
                tribute_id = generate_uuid()
                author = signature_tag.get_text(strip=True)
                tribute_date = date_tag.get_text(strip=True)
                tribute_body_tag = entry.find('div', {'data-qa': 'tribute-summary-message'})
                '''
                if tribute_body_tag:
                    print("Debug: Tribute body tag found")
                else:
                    print("Debug: Tribute body tag not found")
                '''
                tribute_body = tribute_body_tag.get_text(strip=True) if tribute_body_tag else ""
                
                # Store tribute data
                tributes.append({
                    'tribute_id': tribute_id,
                    'author': author,
                    'tribute_date': tribute_date,
                    'tribute_body': tribute_body
                })
            continue
    
        except Exception as e:
            print(f"Error processing entry: {e}")
    
    # Summarize the extraction results
    print(f"Total posts extracted: {len(posts)}")
    print(f"Total tributes extracted: {len(tributes)}")
    
    return {
        'posts': posts,
        'tributes': tributes
    }


In [6]:
# Example usage
try:
    with open('page_source.html', 'r', encoding='utf-8') as f:
        page_source = f.read()
    
    feed_data = extract_feed_data(page_source)
    # Now save the processed data as a pickle file
    with open('feed_data.pkl', 'wb') as pickle_file:
        pickle.dump(feed_data, pickle_file)
    print("Data successfully saved to feed_data.pkl")
except FileNotFoundError:
    print("Error: The file was not found.")
except Exception as e:
    print(f"Unexpected error: {e}")


Total posts extracted: 407
Total tributes extracted: 32
Data successfully saved to feed_data.pkl


feed_data is a dictionary of lists, where each list contains dictionaries representing individual entries.

In [2]:
# To later open the pickled file
'''
try:
    with open('feed_data.pkl', 'rb') as pickle_file:
        feed_data = pickle.load(pickle_file)
    print("Data successfully loaded from feed_data.pkl")
except FileNotFoundError:
    print("Error: Pickle file not found.")
except Exception as e:
    print(f"Error loading pickle file: {e}")
'''

_IncompleteInputError: incomplete input (3795075846.py, line 2)

# Troubleshooting code below

### TROUBLE SHOOTING SECTION TO EXTRACT HTML OF SPECIFIC POSTS - START

In [7]:
def extract_second_post_html(page_source):
    try:
        # Load HTML content using BeautifulSoup
        soup = BeautifulSoup(page_source, 'html.parser')

        # Find all posts using the `article` tag with `data-qa` starting with 'journalPost'
        post_tags = soup.find_all('article', {'data-qa': lambda x: x and x.startswith('journalPost')})

        # Check if there are at least two posts and extract the second one
        if len(post_tags) >= 5:
            second_post_html = post_tags[4].prettify()
            return second_post_html
        else:
            return "Less than two posts found in the feed."

    except Exception as e:
        return f"Error extracting the second post: {e}"

# Example usage
try:
    with open('page_source.html', 'r', encoding='utf-8') as f:
        page_source = f.read()

    second_post_html = extract_second_post_html(page_source)
    print(second_post_html)

except FileNotFoundError:
    print("Error: The file was not found.")
except Exception as e:
    print(f"Unexpected error: {e}")


<article class="jsx-3352288381" data-qa="journalPost undefined" id="bee51c28-dfe6-360f-a9c3-106d10d70ea6">
 <header class="jsx-3352288381 marginTop--0 marginBottom--12">
  <div class="jsx-2074550898 profilePhotoBorder">
   <span class="jsx-1270741811 squareLayout alignTop" data-qa="profile-photo">
    <span class="jsx-1270741811 squareContent">
     <div class="jsx-949504f22dc18149 bkgd">
      <div style="width: 100%; height: 100%; position: relative;">
       <img alt="" data-nimg="1" decoding="async" height="56" loading="lazy" src="https://assets.caringbridge.org/image/upload/c_fill,g_face,ar_1:1,w_56,q_auto,f_auto/legacyPhotos/6e/6ea54ac4-20e9-36d8-a6c0-ec0c8c06993d" style="color: transparent; display: block; object-fit: cover;" width="56"/>
      </div>
     </div>
    </span>
   </span>
   <div class="jsx-2074550898 buttonTopLeft photoButton">
    <img alt="update" aria-hidden="true" class="jsx-3352288381" height="32" src="/img/icons/posts/Update.svg" width="32"/>
   </div>
  </d

In [8]:
import re
# Load HTML using BeautifulSoup
soup = BeautifulSoup(second_post_html, 'html.parser')

# Find the button with the reactionCounter data-qa attribute
print("Finding the button with data-qa that starts with 'reactionCounter'...")
reaction_button = soup.find('button', {'data-qa': lambda x: x and x.startswith('reactionCounter')})
if reaction_button:
    print(f"Reaction button found: {reaction_button}")
else:
    print("Reaction button not found.")

# Find the span within the button that has the reaction count
if reaction_button:
    print("Finding the span with data-qa attribute 'rc-count' within the reaction button...")
    reactions_count_span = reaction_button.find('span', {'data-qa': 'rc-count'})
    if reactions_count_span:
        print(f"Reactions count span found: {reactions_count_span}")
    else:
        print("Reactions count span not found.")
    
    print("Extracting text from the reactions count span...")
    raw_reactions_count_text = reactions_count_span.get_text(strip=True) if reactions_count_span else '0'
    print(f"Raw reactions count text: '{raw_reactions_count_text}'")

    # Extract only the numeric part from the text using a regular expression
    match = re.search(r'\d+', raw_reactions_count_text)
    if match:
        reactions_count = int(match.group())
    else:
        reactions_count = 0

    print(f"Final reactions count: {reactions_count}")
else:
    print("No reaction button found, setting reactions count to 0.")
    reactions_count = 0

print(f"Reactions count: {reactions_count}")


Finding the button with data-qa that starts with 'reactionCounter'...
Reaction button found: <button class="jsx-1378e3855087140a reactionCounter button--isClear" data-qa="reactionCounter-bee51c28-dfe6-360f-a9c3-106d10d70ea6">
<div class="jsx-1378e3855087140a reactionCounter__emoji" data-qa="rc-icon-Heart">
<img alt="Heart icon" class="jsx-1378e3855087140a" src="/img/emojis/emoji-heart.svg"/>
</div>
<div class="jsx-1378e3855087140a reactionCounter__emoji" data-qa="rc-icon-Prayer">
<img alt="Folded hands icon" class="jsx-1378e3855087140a" src="/img/emojis/emoji-pray.svg"/>
</div>
<span class="jsx-1378e3855087140a reactionCounter__count" data-qa="rc-count">
<span class="jsx-1378e3855087140a sr-only">
      Reactions count:
     </span>
     37
    </span>
</button>
Finding the span with data-qa attribute 'rc-count' within the reaction button...
Reactions count span found: <span class="jsx-1378e3855087140a reactionCounter__count" data-qa="rc-count">
<span class="jsx-1378e3855087140a sr-onl

In [9]:
type(feed_data)

dict

In [10]:
feed_data.keys()

dict_keys(['posts', 'tributes'])

In [11]:
for key in feed_data:
    print(len(feed_data[key]))
    

407
32


## Manually inspect posts to make sure that they look right

In [ ]:
feed_data['posts'][:-1]

In [12]:
feed_data['posts'][:3]

[{'post_id': 'bb1aea44-d11b-4da6-88f3-8e3b58ff6330',
  'author': 'Tami Swenson',
  'post_date': 'May 27, 2022',
  'post_title': 'My Story',
  'comments_count': 0,
  'reactions_count': 0,
  'pictures': ['https://assets.caringbridge.org/image/upload/c_fill,g_face,ar_1:1,w_56,q_auto,f_auto/legacyPhotos/16/1639004b-6530-39dd-988c-0e28dca61beb'],
  'post_body': 'Welcome to our CaringBridge website.\xa0 On May 5th 2022 Ada was diagnosed with acute leukemia. We are using it to keep family and friends updated in one place.  We appreciate your support and words of hope and encouragement.  Thank you for visiting.',
  'comments': []},
 {'post_id': 'e7960e13-263d-4eaf-b87d-54bf88e10917',
  'author': 'Joel Swenson',
  'post_date': 'Nov 11, 2024',
  'post_title': 'Monday, November 11, 2024',
  'comments_count': 4,
  'reactions_count': 13,
  'pictures': ['https://assets.caringbridge.org/image/upload/c_fill,g_face,ar_1:1,w_56,q_auto,f_auto/legacyPhotos/6e/6ea54ac4-20e9-36d8-a6c0-ec0c8c06993d',
   'htt

In [16]:
print(json.dumps(feed_data['posts'][-4::], indent=4))

[
    {
        "post_id": "4d731e33-202f-4c61-908c-e467e87cfcf7",
        "author": "Holly Ertmer",
        "post_date": "May 7, 2022",
        "post_title": "May 7, 2022",
        "comments_count": 1,
        "reactions_count": 3,
        "pictures": [
            "https://assets.caringbridge.org/image/upload/c_fill,g_face,ar_1:1,w_145,q_auto,f_auto/legacyPhotos/55ee/55ee8fe7-071d-3528-a0b1-9b252648f1db"
        ],
        "post_body": "",
        "comments": [
            {
                "commenter": "Karen and Mark Swenson",
                "comment_date": "5/11/22",
                "comment_body": "Our Beautiful grandchildren!",
                "reactions_count": 0,
                "replies": []
            }
        ]
    },
    {
        "post_id": "b65352fd-1b94-4ca6-851c-54dc150174bc",
        "author": "Holly Ertmer",
        "post_date": "May 7, 2022",
        "post_title": "May 7, 2022",
        "comments_count": 3,
        "reactions_count": 4,
        "pictures": [
    

In [17]:
print(json.dumps(feed_data['posts'][4]['comments'], indent=4))

[
    {
        "commenter": "Amanda Reed",
        "comment_date": "1/2/24",
        "comment_body": "I live in that district and am willing to help!!Your updates are always so inspiring and often bring me to tears. How you do all that at once as the one navigating loss is amazing.",
        "reactions_count": 1,
        "replies": [
            {
                "reply_author": "Joel Swenson",
                "reply_date": "1/2/24",
                "reply_body": "Thanks Amanda! Are you sure you live in that district? Tami told me that Holly did not live in that district.",
                "reactions_count": 0
            }
        ]
    },
    {
        "commenter": "Jason Anderson",
        "comment_date": "1/2/24",
        "comment_body": "Joel, thank you so much for this update! Sending love to you and the family.",
        "reactions_count": 1,
        "replies": []
    },
    {
        "commenter": "Kristin (Armstrong) DeSouza",
        "comment_date": "12/31/23",
        "comme

In [18]:
print(json.dumps(feed_data['posts'], indent=4))

[
    {
        "post_id": "bb1aea44-d11b-4da6-88f3-8e3b58ff6330",
        "author": "Tami Swenson",
        "post_date": "May 27, 2022",
        "post_title": "My Story",
        "comments_count": 0,
        "reactions_count": 0,
        "pictures": [
            "https://assets.caringbridge.org/image/upload/c_fill,g_face,ar_1:1,w_56,q_auto,f_auto/legacyPhotos/16/1639004b-6530-39dd-988c-0e28dca61beb"
        ],
        "post_body": "Welcome to our CaringBridge website.\u00a0 On May 5th 2022 Ada was diagnosed with acute leukemia. We are using it to keep family and friends updated in one place.  We appreciate your support and words of hope and encouragement.  Thank you for visiting.",
        "comments": []
    },
    {
        "post_id": "e7960e13-263d-4eaf-b87d-54bf88e10917",
        "author": "Joel Swenson",
        "post_date": "Nov 11, 2024",
        "post_title": "Monday, November 11, 2024",
        "comments_count": 4,
        "reactions_count": 13,
        "pictures": [
        

## Manually inspect tributes to make sure that they look right

In [19]:
feed_data['tributes'][-2::]

[{'tribute_id': '2c5c054b-733f-4b28-ae33-c25a9a40e7c1',
  'author': 'Mark Swenson',
  'tribute_date': 'May 8, 2022',
  'tribute_body': ''},
 {'tribute_id': '4d1a12db-df2e-48c1-889d-fbdf61777f0f',
  'author': 'Thinking Of You Sweet Ada! Hugs And Prayers! Love Ron And Patty',
  'tribute_date': 'May 8, 2022',
  'tribute_body': ''}]

In [20]:
print(json.dumps(feed_data['tributes'][-4::], indent=4))

[
    {
        "tribute_id": "239acd7c-1053-4f8a-8c4d-db00c682c87d",
        "author": "Cheryl and Tim Clark",
        "tribute_date": "May 8, 2022",
        "tribute_body": ""
    },
    {
        "tribute_id": "fd2ae94b-22a4-4a41-97ea-ff19093bd6e5",
        "author": "Patti Spaid (Felker)",
        "tribute_date": "May 8, 2022",
        "tribute_body": ""
    },
    {
        "tribute_id": "2c5c054b-733f-4b28-ae33-c25a9a40e7c1",
        "author": "Mark Swenson",
        "tribute_date": "May 8, 2022",
        "tribute_body": ""
    },
    {
        "tribute_id": "4d1a12db-df2e-48c1-889d-fbdf61777f0f",
        "author": "Thinking Of You Sweet Ada! Hugs And Prayers! Love Ron And Patty",
        "tribute_date": "May 8, 2022",
        "tribute_body": ""
    }
]


In [24]:
print(json.dumps(feed_data['tributes'][4]['comments'], indent=4))

KeyError: 'comments'

In [22]:
print(json.dumps(feed_data['tributes'], indent=4))

[
    {
        "tribute_id": "fbc896f2-d6e3-4bb8-ba94-6521779d7ab7",
        "author": "M. Zeman",
        "tribute_date": "December 31, 2023",
        "tribute_body": ""
    },
    {
        "tribute_id": "0639208e-1b23-4e36-811b-a91be1e35e59",
        "author": "Janet Gauger",
        "tribute_date": "August 29, 2023",
        "tribute_body": ""
    },
    {
        "tribute_id": "5cd774c6-48da-42dd-8c41-157e0f061474",
        "author": "Mary Roelke",
        "tribute_date": "July 24, 2023",
        "tribute_body": ""
    },
    {
        "tribute_id": "a9f66f3a-f41c-4928-84dd-b6cfd62929e1",
        "author": "Eyal Halamish",
        "tribute_date": "July 21, 2023",
        "tribute_body": "Hi Joel,   It's been a while, but I wanted to send you lots of love and support.  Love your old roommate,"
    },
    {
        "tribute_id": "11ab9327-4eb3-4d66-acbe-96c350a2f275",
        "author": "Mindy, Charan, Rohan & Ari",
        "tribute_date": "July 3, 2023",
        "tribute_body": "Se

## Understanding the structure of feed_data

In [25]:
print(type(feed_data))  # Should print: <class 'dict'>
print(type(feed_data['posts']))  # Should print: <class 'list'>
print(type(feed_data['tributes']))  # Should print: <class 'list'>

<class 'dict'>
<class 'list'>
<class 'list'>


In [26]:
# Print type of feed_data
print("Type of feed_data:", type(feed_data))  # Should print: <class 'dict'>

# Print type of feed_data['posts']
print("Type of feed_data['posts']:", type(feed_data['posts']))  # Should print: <class 'list'>

# Print type of elements within feed_data['posts']
if feed_data['posts']:
    print("Type of an element in feed_data['posts']:", type(feed_data['posts'][0]))  # Should print: <class 'dict'>
else:
    print("feed_data['posts'] is empty")

# Print type of feed_data['tributes']
print("Type of feed_data['tributes']:", type(feed_data['tributes']))  # Should print: <class 'list'>

# Print type of elements within feed_data['tributes']
if feed_data['tributes']:
    print("Type of an element in feed_data['tributes']:", type(feed_data['tributes'][0]))  # Should print: <class 'dict'>
else:
    print("feed_data['tributes'] is empty")

print("Thus, feed_data is a dictionary of lists, where each list contains dictionaries representing individual entries.")

Type of feed_data: <class 'dict'>
Type of feed_data['posts']: <class 'list'>
Type of an element in feed_data['posts']: <class 'dict'>
Type of feed_data['tributes']: <class 'list'>
Type of an element in feed_data['tributes']: <class 'dict'>
Thus, feed_data is a dictionary of lists, where each list contains dictionaries representing individual entries.


In [8]:
print(json.dumps(feed_data['posts'][4], indent=4))

{
    "post_id": "87aff84e-71b2-4d59-9ae0-6787707014be",
    "author": "Joel Swenson",
    "post_date": "Dec 31, 2023",
    "post_title": "December 31, 2023 - Update and request for help from our Appleton/Neenah friends",
    "comments_count": 13,
    "reactions_count": 37,
    "pictures": [
        "https://assets.caringbridge.org/image/upload/c_fill,g_face,ar_1:1,w_56,q_auto,f_auto/legacyPhotos/6e/6ea54ac4-20e9-36d8-a6c0-ec0c8c06993d",
        "https://assets.caringbridge.org/image/upload/c_fill,g_face,ar_1:1,w_145,q_auto,f_auto/legacyPhotos/55dd/55dd6f5b-c3d0-3c96-81f2-48799fdfa251",
        "https://assets.caringbridge.org/image/upload/c_fill,g_face,ar_1:1,w_145,q_auto,f_auto/legacyPhotos/3626/36269f54-e1f0-30cc-bfab-c4b040b7a0fb",
        "https://assets.caringbridge.org/image/upload/c_fill,g_face,ar_1:1,w_145,q_auto,f_auto/legacyPhotos/9d78/9d780d39-024f-3553-8e09-bcb46c398208",
        "https://assets.caringbridge.org/image/upload/c_fill,g_face,ar_1:1,w_145,q_auto,f_auto/legacyP

In [27]:
print(json.dumps(feed_data['tributes'][:2], indent=4))

[
    {
        "tribute_id": "fbc896f2-d6e3-4bb8-ba94-6521779d7ab7",
        "author": "M. Zeman",
        "tribute_date": "December 31, 2023",
        "tribute_body": ""
    },
    {
        "tribute_id": "0639208e-1b23-4e36-811b-a91be1e35e59",
        "author": "Janet Gauger",
        "tribute_date": "August 29, 2023",
        "tribute_body": ""
    }
]


ok, I want to port the data into a SQL database that uses a snowflake schema. Here is what the data structure looks like, please propose a schema.

print(json.dumps(feed_data['tributes'][:2], indent=4))
[
    {
        "tribute_id": "fefafdcf-44e0-4e98-b1ff-2b11d380825e",
        "author": "M. Zeman",
        "tribute_date": "December 31, 2023",
        "tribute_body": ""
    },
    {
        "tribute_id": "a56de4bc-3074-4cb5-82a5-b8a62530df03",
        "author": "Janet Gauger",
        "tribute_date": "August 29, 2023",
        "tribute_body": ""
    }
]

print(json.dumps(feed_data['posts'][4], indent=4))

{
    "post_id": "87aff84e-71b2-4d59-9ae0-6787707014be",
    "author": "Joel Swenson",
    "post_date": "Dec 31, 2023",
    "post_title": "December 31, 2023 - Update and request for help from our Appleton/Neenah friends",
    "comments_count": 13,
    "reactions_count": 37,
    "pictures": [
        "https://assets.caringbridge.org/image/upload/c_fill,g_face,ar_1:1,w_56,q_auto,f_auto/legacyPhotos/6e/6ea54ac4-20e9-36d8-a6c0-ec0c8c06993d",
        "https://assets.caringbridge.org/image/upload/c_fill,g_face,ar_1:1,w_145,q_auto,f_auto/legacyPhotos/55dd/55dd6f5b-c3d0-3c96-81f2-48799fdfa251",
        "https://assets.caringbridge.org/image/upload/c_fill,g_face,ar_1:1,w_145,q_auto,f_auto/legacyPhotos/3626/36269f54-e1f0-30cc-bfab-c4b040b7a0fb",
        "https://assets.caringbridge.org/image/upload/c_fill,g_face,ar_1:1,w_145,q_auto,f_auto/legacyPhotos/9d78/9d780d39-024f-3553-8e09-bcb46c398208",
        "https://assets.caringbridge.org/image/upload/c_fill,g_face,ar_1:1,w_145,q_auto,f_auto/legacyPhotos/16b9/16b9d653-e898-3e01-87b3-afae5e569680",
        "https://assets.caringbridge.org/image/upload/c_fill,g_face,ar_1:1,w_145,q_auto,f_auto/legacyPhotos/6e34/6e34b95a-779d-3da0-96f6-22a3d7d5ecc5",
        "https://assets.caringbridge.org/image/upload/c_fill,g_face,ar_1:1,w_40,q_auto,f_auto/legacyPhotos/6e/6ea54ac4-20e9-36d8-a6c0-ec0c8c06993d",
        "https://assets.caringbridge.org/image/upload/c_fill,g_face,ar_1:1,w_40,q_auto,f_auto/legacyPhotos/cb/cb2cef17-9bd1-31a6-879f-be953f9b0cbc",
        "https://assets.caringbridge.org/image/upload/c_fill,g_face,ar_1:1,w_40,q_auto,f_auto/legacyPhotos/6e/6ea54ac4-20e9-36d8-a6c0-ec0c8c06993d",
        "https://assets.caringbridge.org/image/upload/c_fill,g_face,ar_1:1,w_40,q_auto,f_auto/legacyPhotos/0c/0cffd12a-01c3-396b-a2dc-08f92c72e99a",
        "https://assets.caringbridge.org/image/upload/c_fill,g_face,ar_1:1,w_40,q_auto,f_auto/legacyPhotos/6e/6ea54ac4-20e9-36d8-a6c0-ec0c8c06993d",
        "https://assets.caringbridge.org/image/upload/c_fill,g_face,ar_1:1,w_40,q_auto,f_auto/legacyPhotos/6e/6ea54ac4-20e9-36d8-a6c0-ec0c8c06993d"
    ],
    "post_body": "Hi all,I'm not sure if folks still monitor/get alerts here but wanted to share a quick update and also get some help trying to pass a bill funding childhood cancer research.Request for help if you live in Appleton or Neenah:Tami and I are both getting involved with various organizations to help families currently battling childhood cancer or leukemia. We know how devastating cancer can be and want to do our part. I'm volunteering (just got started basically!) with the MACC and ACCO organizations and we are helping introduce a bill into the Wisconsin State Legislature that establishes a state administered trust that would exclusively fund pediatric cancer reseove.Show more",
    "comments": [
        {
            "commenter": "Amanda Reed",
            "comment_date": "1/2/24",
            "comment_body": "I live in that district and am willing to help!!Your updates are always so inspiring and often bring me to tears. How you do all that at once as the one navigating loss is amazing.",
            "reactions_count": 1,
            "replies": [
                {
                    "reply_author": "Joel Swenson",
                    "reply_date": "1/2/24",
                    "reply_body": "Thanks Amanda! Are you sure you live in that district? Tami told me that Holly did not live in that district.",
                    "reactions_count": 0
                }
            ]
        },
        {
            "commenter": "Jason Anderson",
            "comment_date": "1/2/24",
            "comment_body": "Joel, thank you so much for this update! Sending love to you and the family.",
            "reactions_count": 1,
            "replies": []
        },
        {
            "commenter": "Kristin (Armstrong) DeSouza",
            "comment_date": "12/31/23",
            "comment_body": "Joel, Tami, Oliver, and Ellie, you continue to set an amazing example of beautiful ways to navigate such an experience by honoring and remembering Ada in such beautiful ways that represent her so well. Thank you for sharing. Sending love to you all, Kristin \u2665\ufe0f",
            "reactions_count": 1,
            "replies": []
        },
        {
            "commenter": "Jane Klimenko",
            "comment_date": "12/31/23",
            "comment_body": "Sending our loving thoughts your way. May you continue holding each other in your arms and in your hearts in the New Year and many years more \u2764\ufe0f Jane and family",
            "reactions_count": 1,
            "replies": []
        },
        {
            "commenter": "Lance Paulson",
            "comment_date": "12/31/23",
            "comment_body": "Tami, Joel and kids, we are amazed by the example you repeatedly set for how a family can navigate this journey with love, strength and humility and then continue to honor Ada by giving back to the cause.  We look forward to seeing you at ice skating!",
            "reactions_count": 1,
            "replies": []
        },
        {
            "commenter": "Maureen Heider",
            "comment_date": "12/31/23",
            "comment_body": "Joel & Tami,This post gave me hope for our future. That I will be able to find joy in memories-right now it\u2019s tsunami sized waves of emotions.Thank you for the toy drive for  AFCH-it will bring smiles to brave kiddos facing things most adults have problems dealing with! It made me smile remembering Ada riding her trike and the sticky balls stuck to the ceiling at AFCH :)Tami, you are amazing going back to work with pharmaceutical products that you have personally knowledge about.Joel, being a rock for Oliver & Ellie is absolutely wonderful.The way both of you are taking care of your family is amazing.May 2024 be a year of healing and peace.Show more",
            "reactions_count": 1,
            "replies": [
                {
                    "reply_author": "Joel Swenson",
                    "reply_date": "12/31/23",
                    "reply_body": "We think of Amelia and your family often. Amelia was always so sweet towards Ada. Wishing you and your family peace in 2024 and please reach out to us if we can help at all. We've found a few good organizations to help families with grief, email me if you are interested, I know some people don't want that right away. Joel.swenson at Gmail dot comShow more",
                    "reactions_count": 1
                }
            ]
        },
        {
            "commenter": "Nicholas Torres",
            "comment_date": "12/31/23",
            "comment_body": "\u2764\ufe0f",
            "reactions_count": 1,
            "replies": []
        },
        {
            "commenter": "Ally Turner",
            "comment_date": "12/31/23",
            "comment_body": "You are both so strong! I think about your family often, and think fondly of Ada when I see at my smiley face tattoo that I got in her memory. She made such an impact on me, as I know she did on so many! Her spirit was definitely in my heart this holiday season. Sending all my love to your family. Happy holidays, and Happy New Year. Let 2024 be a year of love and life and joy \u2764\ufe0fShow more",
            "reactions_count": 1,
            "replies": [
                {
                    "reply_author": "Joel Swenson",
                    "reply_date": "12/31/23",
                    "reply_body": "Thank you! Would love to see a picture of the tattoo sometime!",
                    "reactions_count": 0
                }
            ]
        },
        {
            "commenter": "Elise Hansen",
            "comment_date": "12/31/23",
            "comment_body": "Joel and Tami,As I read this I am with our good friends that I told you about who also lost a young child when he was age 11. They said to tell you that you will aways remember, grieve and miss Ada everyday, that will never change. However, they say that the memories are special and  you will eventually appreciate that you have these special loving memories. They recommend you laugh, hug, love , create some new traditions and enjoy the memories! (even 24 years later.)With all of our love,Elise and friendsShow more",
            "reactions_count": 2,
            "replies": [
                {
                    "reply_author": "Joel Swenson",
                    "reply_date": "12/31/23",
                    "reply_body": "Thank you! We will try our best!",
                    "reactions_count": 0
                }
            ]
        }
    ]
}

-- Authors Dimension
CREATE TABLE dim_authors (
    author_id VARCHAR(255) PRIMARY KEY,
    author_name VARCHAR(255)
);

-- Date Dimension
CREATE TABLE dim_dates (
    date_id VARCHAR(255) PRIMARY KEY,
    full_date DATE,
    year INT,
    month VARCHAR(50),
    day INT
);

-- Image Dimension 
CREATE TABLE dim_images (
    image_id VARCHAR(255) PRIMARY KEY,
    image_url TEXT
);

In [ ]:
-- Tributes Fact Table
CREATE TABLE fact_tributes (
    tribute_id VARCHAR(255) PRIMARY KEY,
    author_id VARCHAR(255),
    date_id VARCHAR(255),
    tribute_body TEXT,
    FOREIGN KEY (author_id) REFERENCES dim_authors(author_id),
    FOREIGN KEY (date_id) REFERENCES dim_dates(date_id)
);

-- Posts Fact Table
CREATE TABLE fact_posts (
    post_id VARCHAR(255) PRIMARY KEY,
    author_id VARCHAR(255),
    date_id VARCHAR(255),
    post_title TEXT,
    post_body TEXT,
    comments_count INT,
    reactions_count INT,
    FOREIGN KEY (author_id) REFERENCES dim_authors(author_id),
    FOREIGN KEY (date_id) REFERENCES dim_dates(date_id)
);

-- Comments Fact Table
CREATE TABLE fact_comments (
    comment_id VARCHAR(255) PRIMARY KEY,
    post_id VARCHAR(255),
    commenter_id VARCHAR(255),
    date_id VARCHAR(255),
    comment_body TEXT,
    reactions_count INT,
    FOREIGN KEY (post_id) REFERENCES fact_posts(post_id),
    FOREIGN KEY (commenter_id) REFERENCES dim_authors(author_id),
    FOREIGN KEY (date_id) REFERENCES dim_dates(date_id)
);

-- Replies Fact Table
CREATE TABLE fact_replies (
    reply_id VARCHAR(255) PRIMARY KEY,
    comment_id VARCHAR(255),
    reply_author_id VARCHAR(255),
    date_id VARCHAR(255),
    reply_body TEXT,
    reactions_count INT,
    FOREIGN KEY (comment_id) REFERENCES fact_comments(comment_id),
    FOREIGN KEY (reply_author_id) REFERENCES dim_authors(author_id),
    FOREIGN KEY (date_id) REFERENCES dim_dates(date_id)
);

-- Post Images Bridge Table
CREATE TABLE bridge_post_images (
    post_id VARCHAR(255),
    image_id VARCHAR(255),
    PRIMARY KEY (post_id, image_id),
    FOREIGN KEY (post_id) REFERENCES fact_posts(post_id),
    FOREIGN KEY (image_id) REFERENCES dim_images(image_id)
);